In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis= 1, inplace= True)

## **Data Splitting**

In [7]:
from sklearn.model_selection import train_test_split

X = df.drop('Exited', axis= 1)
y = df.Exited

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42, stratify= y)

In [8]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_cols = df.select_dtypes(include= np.number).columns.to_list()
categorical_cols = df.select_dtypes(exclude= np.number).columns.to_list()

In [9]:
encoder = OneHotEncoder(sparse_output= False)

train_encoder = encoder.fit_transform(X_train[categorical_cols])
test_encoder = encoder.fit_transform(X_test[categorical_cols])

encoded_names = encoder.get_feature_names_out()

In [10]:
merged_train = pd.DataFrame(
    train_encoder,
    columns= encoded_names
)

In [11]:
merged_test = pd.DataFrame(
    test_encoder,
    columns= encoded_names
)

In [12]:
merged_train.head()

,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0


## **Scalling numeric features**

In [13]:
numeric_cols.pop()

'Exited'

In [14]:
X_train = pd.concat([X_train, merged_train], axis= 1)
X_test = pd.concat([X_test, merged_test], axis= 1)

In [15]:
X_train.drop(categorical_cols, axis= 1, inplace= True)
X_test.drop(categorical_cols, axis= 1, inplace=True)

In [16]:
X_train.dropna(inplace=True)

In [17]:
scaler = StandardScaler()


X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

## **Model Building and Evaluation**

In [18]:
y_train.value_counts()

Exited
0    6370
1    1630
Name: count, dtype: int64

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [20]:
model_dict = {
    'lr': LogisticRegression(max_iter= 1000, random_state= 42),
    'dt': DecisionTreeClassifier(random_state=42),
    'rf': RandomForestClassifier(random_state= 42)
}

In [21]:
accuracy_per_model = {}

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [26]:
len(X_train), len(y_train)

(6385, 8000)

In [25]:
for name, model in model_dict.items():
    models = model
    models.fit(X_train, y_train)

    y_pred = models.predict(X_test)
    print(accuracy_score(y_test, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [6385, 8000]